In [ ]:
import pandas as pd
import numpy as np
import re
import os

def clean(n):
    n = ''.join(n.split())
    n = n.replace('n|','').replace('ic-','').replace('stx-','').replace('[KoG]^','').replace(']km[','')
    n = n.replace('[B]','').replace('territory.ru>','').replace('esp-','').replace('ss-','').replace('K9-','')
    n = n.replace('socrates_','socrates').replace('ASUS*c58-','').replace('ve-','').replace('g1|','')
    n = n.replace('TOXIC','toxjq').replace('.','').replace('territoryru`','')
    return n
def clean2(x):
    for sub in ['cK-','All*','{DigA}','c4']:
        x = x.replace(sub,'')
    return x

In [ ]:
FILE = 'na1.txt'
TYPE = 'se'

In [ ]:
brack_text = {}
for t in os.listdir('brackets'):
    brack_text[t] = open('brackets/' + t).read()
    
import pickle
with open('flags.pkl','rb') as fp:
    flags = pickle.load(fp)
flags['pow3r'] = 'ru'
flags['slinger'] = 'de'
flags['treble'] = 'se'
flags['zyz'] = 'de'
flags['polosatiy'] = 'ru'
flags['powerk'] = 'kr'
flags['neok'] = 'de'
flags['shurik'] = 'ru'
flags['guard'] = 'lt'
flags['p0ni'] = 'es'

In [ ]:
with open(FILE) as fp:
    text = fp.read()
    
text_games = [_.split('\n') for _ in text.split('\n\n')]


all_matches = []
for sec in text.split('\n\n'):
    sec = sec.replace('\n\n','\n')
    matches = []
    s1,s2 = -1,-1
    #match_cand = re.findall('(.+?)\s+([><]|vs)\s+(.+?)[-\s]+([\d-]+):([-\d]+)',sec)
    match_cand = re.findall('(.+?) (.+?) ([\d-]+) ([\d-]+)',sec)
    #match_cand = re.findall('(.+) \[(\d+):(\d+)\] (.+)',sec)
    #match_cand = re.findall('(.+) \[ (\d+):(\d+) \] (.+)',sec)

    for p1,p2,s1,s2 in match_cand:
        p1 = clean(p1)
        p2 = clean(p2)
        s1 = int(s1)
        s2 = int(s2)
        matches.append((p1,p2,s1,s2))
    #match_cand = re.findall('(.+?)\s+([><]|vs)\s+([^:]+?)\n',sec)
    all_matches.append(matches)
    
all_matches

In [ ]:
codes = brack_codes[str(2*len(all_matches[0])) + TYPE + '.txt']

In [ ]:
[len(_) for _ in all_matches],sum([len(_) for _ in all_matches])
decon_idx = -1
plen = 128
for i in range(len(all_matches)):
    if len(all_matches[i]) > plen:
        decon_idx = i
        break
    else:
        plen = len(all_matches[i])
if decon_idx != -1:
    top_brack = all_matches[:decon_idx]
    bot_brack = all_matches[decon_idx:]
else:
    top_brack = all_matches
    bot_brack = []

In [ ]:
[len(_) for _ in top_brack]

In [ ]:

[len(_) for _ in bot_brack]


In [ ]:
tres = []
for brack in [_ for _ in [top_brack,bot_brack] if len(_) > 0]:
    res = []

    queue = [list(_) + [0] for _ in brack[-1]]
    all_match_us = sum(brack[:-1],[])[::-1]

    temp_res = []
    curr_level = 0
    print(queue,res)
    while len(queue) > 0:
        print(len(queue),len(res),curr_level, len(temp_res),len(all_match_us))

        match=queue.pop(0)
        flip = False
        if match[-1] != curr_level:
            curr_level += 1
            res.append(temp_res)
            temp_res = []
        del_idx = -1
        for i in range(len(all_match_us)):
            if all_match_us[i][0] == match[0] or all_match_us[i][1] == match[0]:
                queue.append(list(all_match_us[i]) + [curr_level+1])
                del_idx = i
                break
        #print(del_idx,len(all_match_us))
        if del_idx != -1:
            del all_match_us[del_idx]
            flip =True
        del_idx = -1
        for i in range(len(all_match_us)):
            if all_match_us[i][0] == match[1] or all_match_us[i][1] == match[1]:
                queue.append(list(all_match_us[i]) + [curr_level+1])
                del_idx = i
                break
        #print(del_idx,len(all_match_us))
        if del_idx != -1:
            del all_match_us[del_idx]
            flip = False
        if flip:
            match = (match[1],match[0],match[3],match[2],match[4])
        temp_res.append(match)
    #temp_res.append(['schnucks', 'SoulXtort', 0, 0] + [curr_level])
    res.append(temp_res)
    tres.append(res)

In [ ]:
[[len(x) for x in _ ] for _ in tres]

In [ ]:
tres

In [ ]:
if False:
    tres[1][-1].insert(3,['bye','bye',0,0,7])
    tres[1][-1].insert(0,['bye','bye',0,0,7])
    tres[1][-1].append(['bye','bye',0,0,7])
    tres[1][-1]

In [ ]:
new_res = sum([_[::-1] for _ in tres],[])
new_res = sum(new_res,[])
#|R1D43=belmakor|R1D43score=15 |R1D43flag= |R1D43win=1
#|R1D44=dCypher|R1D44score=7 |R1D44flag= |R1D44win=
for i in range(0,len(new_res)):
    p1,p2,s1,s2,lvl = new_res[i]
    s1,s2 = int(s1),int(s2)
    w1 = '1' if int(s1) > int(s2) else ''
    w2 = '1' if int(s2) > int(s1) else ''
    bn =codes[2*i]
    if abs(s1) + abs(s2) ==1:
        s1 = ''
        s2 = ''
    f1 = flags.get(p1.lower(),'')
    f2 = flags.get(p2.lower(),'')
    if p2 == 'bye':
        f2 = ''

    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p1,bn,s1,bn,f1,bn,w1))
    bn = codes[2*i+1]
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p2,bn,s2,bn,f2,bn,w2))
print()
for i in range(i+1,len(codes)//2):
    p1,p2 = '',''
    s1,s2 ='',''
    w1 = ''
    w2 = ''
    bn =codes[2*i]
    f1 = ''#flags.get(p1.lower(),'ru')
    f2 = ''
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p1,bn,s1,bn,f1,bn,w1))
    bn = codes[2*i+1]
    print('|{}={}|{}score={} |{}flag={} |{}win={}'.format(bn,p2,bn,s2,bn,f2,bn,w2))


In [ ]:
all_matches

In [ ]:
len(codes)/2,len(res),print(all_match_us)